In [1]:
import os
import sys
import json
from pathlib import Path

def check_environment():
    """檢查環境設定並輸出相關資訊"""

    print("=== Python 環境資訊 ===")
    print(f"Python 版本: {sys.version}")
    print(f"Python 執行路徑: {sys.executable}")
    print(f"目前工作目錄: {os.getcwd()}")

    print("\n=== PYTHONPATH 設定 ===")
    python_path = os.getenv('PYTHONPATH', '未設定')
    print(f"PYTHONPATH: {python_path}")

    print("\n=== VS Code 設定檔檢查 ===")
    vscode_settings_path = Path('.vscode/settings.json')
    if vscode_settings_path.exists():
        with open(vscode_settings_path, 'r', encoding='utf-8') as f:
            settings = json.load(f)
            print("找到 VS Code 設定檔:")
            print(f"- 預設 Python 路徑: {settings.get('python.defaultInterpreterPath', '未設定')}")
            print(f"- Python 格式化工具: {settings.get('[python]', {}).get('editor.defaultFormatter', '未設定')}")
    else:
        print("未找到 VS Code 設定檔")

    print("\n=== 已安裝的相關套件 ===")
    try:
        import black
        print("✓ black-formatter 已安裝")
    except ImportError:
        print("✗ black-formatter 未安裝")

    try:
        import pylint
        print("✓ pylint 已安裝")
    except ImportError:
        print("✗ pylint 未安裝")

if __name__ == "__main__":
    check_environment()

=== Python 環境資訊 ===
Python 版本: 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]
Python 執行路徑: c:\Program Files\Python310\python.exe
目前工作目錄: c:\Users\xdxd2\Documents\GitHub\iSpan_LLM-NLP-cookbooks\Langchain_scratch\langchain_framework\Course\Module1

=== PYTHONPATH 設定 ===
PYTHONPATH: 未設定

=== VS Code 設定檔檢查 ===
未找到 VS Code 設定檔

=== 已安裝的相關套件 ===
✓ black-formatter 已安裝
✓ pylint 已安裝


In [5]:
from pathlib import Path

def fix_env_file():
    """移除 .env 檔案中的 BOM 標記"""
    env_path = r"C:\Users\xdxd2\Documents\GitHub\iSpan_LLM-NLP-cookbooks\Langchain_scratch\langchain_framework\.env"

    # 讀取檔案內容
    with open(env_path, 'r', encoding='utf-8-sig') as f:
        content = f.read()

    # 使用 utf-8 編碼重新寫入（不含 BOM）
    with open(env_path, 'w', encoding='utf-8') as f:
        f.write(content)

    print(f"已修復 {env_path} 的編碼")

    # 顯示修復後的內容
    with open(env_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        print("\n修復後的環境變數:")
        for line in lines:
            if '=' in line and not line.startswith('#'):
                var_name = line.split('=')[0].strip()
                print(f"- {var_name}")

if __name__ == "__main__":
    fix_env_file()

已修復 C:\Users\xdxd2\Documents\GitHub\iSpan_LLM-NLP-cookbooks\Langchain_scratch\langchain_framework\.env 的編碼

修復後的環境變數:
- OPENAI_API_KEY
- ANTHROPIC_API_KEY
- TAVILY_API_KEY
- LANG_SMITH_KEY


In [1]:
"""
LangChain 0.3+ Agent 自動化教學
展示如何建立自動化工作流程，包含任務規劃、執行追蹤和錯誤處理

需求套件:
- langchain>=0.3.0
- langchain-openai>=0.0.2
- langchain-community>=0.0.1
- python-dotenv>=0.19.0
- tenacity>=8.0.1
"""

from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
# from typing_extensions import Annotated
from typing import List, Dict, Any, Annotated
from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_exponential
import logging
import os
import json
import datetime

# 設定日誌
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# 載入環境變數
load_dotenv()


class TaskPlan(BaseModel):
    """任務規劃結構"""
    steps: List[str] = Field(description="執行步驟列表")
    tools_needed: List[str] = Field(description="需要使用的工具列表")
    estimated_time: str = Field(description="預估完成時間")


class TaskResult(BaseModel):
    """任務結果結構"""
    success: bool = Field(description="是否成功完成")
    result: str = Field(description="執行結果")
    error: str = Field(default="", description="錯誤訊息")


@tool
def save_to_file(content: str, filename: str) -> str:
    """
    將內容保存到檔案

    Args:
        content: 要保存的內容
        filename: 檔案名稱
    """
    try:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)
        return f"成功保存到檔案: {filename}"
    except Exception as e:
        return f"保存失敗: {str(e)}"


@tool
def read_from_file(filename: str) -> str:
    """
    從檔案讀取內容

    Args:
        filename: 檔案名稱
    """
    try:
        with open(filename, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"讀取失敗: {str(e)}"


@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, # 基本等待時間（秒）
                          min=4, # 最小等待時間
                          max=10 # 最大等待時間
                          )
)
@tool
def process_task(task_description: str) -> str:
    """
    處理任務（示例：模擬耗時操作）

    Args:
        task_description: 任務描述
    """
    # 這裡可以加入實際的任務處理邏輯
    return f"已完成任務: {task_description}"


def create_automation_agent() -> AgentExecutor:
    """
    建立自動化 Agent
    """
    # 建立 LLM
    llm = ChatOpenAI(
        temperature=0,
        model="gpt-3.5-turbo"
    )

    print("check")

    # 建立輸出解析器
    plan_parser = PydanticOutputParser(pydantic_object=TaskPlan)
    result_parser = PydanticOutputParser(pydantic_object=TaskResult)

    # 定義系統提示詞
    system_prompt = """你是一位自動化助理，負責規劃和執行任務。

    你可以：
    1. 保存內容到檔案
    2. 從檔案讀取內容
    3. 處理指定的任務

    執行任務時，請：
    1. 先規劃任務步驟，並以 JSON 格式輸出，包含：
       - steps: 執行步驟列表
       - tools_needed: 需要使用的工具列表
       - estimated_time: 預估完成時間
    2. 按步驟執行
    3. 記錄執行結果，並以 JSON 格式輸出，包含：
       - success: 是否成功完成
       - result: 執行結果
       - error: 錯誤訊息（如果有）

    回答時請使用繁體中文，並保持專業、友善的態度。

    {plan_format_instructions}
    {result_format_instructions}
    """

    # 建立提示詞模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt.format(
            plan_format_instructions=plan_parser.get_format_instructions(),
            result_format_instructions=result_parser.get_format_instructions()
        )),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])

    # 建立工具列表
    tools = [
        save_to_file,
        read_from_file,
        process_task
    ]

    # 建立 Agent
    agent = create_openai_functions_agent(llm, tools, prompt)

    # 建立 Agent 執行器
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True
    )

    return agent_executor


def demonstrate_automation():
    """
    展示自動化任務處理
    """
    agent_executor = create_automation_agent()
    chat_history = []

    # 建立輸出解析器
    plan_parser = PydanticOutputParser(pydantic_object=TaskPlan)
    result_parser = PydanticOutputParser(pydantic_object=TaskResult)


    # 測試案例
    test_cases = [
        "請將'這是測試內容'保存到 test.txt，然後讀取並確認內容",
        "請執行一個任務：整理當前目錄下的檔案",
        "請規劃一個多步驟任務：1.創建檔案 2.寫入內容 3.讀取確認 4.輸出結果",
        "請處理以下任務並記錄結果：分析一段文本的關鍵字"
    ]

    for i, task in enumerate(test_cases, 1):
        print(f"\n=== 測試案例 {i} ===")
        print(f"任務: {task}")
        try:
            # 執行任務並解析結果
            response = agent_executor.invoke({
                "input": task,
                "chat_history": chat_history
            })

            try:
                # 嘗試解析任務計劃和結果
                task_plan = plan_parser.parse(response["output"])
                print("\n任務計劃:")
                print(f"步驟: {task_plan.steps}")
                print(f"需要工具: {task_plan.tools_needed}")
                print(f"預估時間: {task_plan.estimated_time}")

                task_result = result_parser.parse(response["output"])
                print("\n執行結果:")
                print(f"成功: {task_result.success}")
                print(f"結果: {task_result.result}")
                if task_result.error:
                    print(f"錯誤: {task_result.error}")
            except Exception as parse_error:
                logger.warning(f"解析結果失敗: {str(parse_error)}")
                print(f"原始回應: {response['output']}")

            # 更新對話歷史
            chat_history.extend([
                ("human", task),
                ("assistant", response["output"])
            ])
        except Exception as e:
            logger.error(f"執行失敗: {str(e)}")


def main():
    """
    主程式：展示 Agent 自動化功能
    """
    print("=== LangChain 0.3+ Agent 自動化展示 ===\n")

    if not os.getenv("OPENAI_API_KEY"):
        logger.error("請先設定 OPENAI_API_KEY 環境變數！")
        return

    try:
        demonstrate_automation()
    except Exception as e:
        logger.error(f"執行過程發生錯誤: {str(e)}")


if __name__ == "__main__":
    main()


2025-01-31 13:09:27,748 - ERROR - 執行過程發生錯誤: name 'Annotated' is not defined


=== LangChain 0.3+ Agent 自動化展示 ===

check


In [3]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

from typing import Optional

from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

2025-01-31 13:13:53,960 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!',
 'rating': 7}

In [16]:
from pydantic import BaseModel, Field
from typing import List
from typing_extensions import Annotated  # 適用於 Python 3.8

class TaskPlan(BaseModel):
    """任務規劃結構"""
    steps: List[str] = Field(description="執行步驟列表")
    tools_needed: List[str] = Field(description="需要使用的工具列表")
    estimated_time: Annotated[str, Field(description="預估完成時間")]

# 建立任務規劃的實例
task_plan = TaskPlan(
    steps=["寫報告", "審核報告", "發送電子郵件"],
    tools_needed=["Microsoft Word", "Gmail"],
    estimated_time="2 小時"
)

# 顯示結果
print(task_plan.model_dump_json(indent=2))




{
  "steps": [
    "寫報告",
    "審核報告",
    "發送電子郵件"
  ],
  "tools_needed": [
    "Microsoft Word",
    "Gmail"
  ],
  "estimated_time": "2 小時"
}


In [17]:
from langchain.output_parsers import PydanticOutputParser

# 初始化 PydanticOutputParser
plan_parser = PydanticOutputParser(pydantic_object=TaskPlan)

# ✅ 讓 PydanticOutputParser 解析 JSON 字串
parsed_plan = plan_parser.parse(task_plan.model_dump_json())

# ✅ 驗證解析後的輸出
print(parsed_plan)
print(type(parsed_plan))  # 應該是 TaskPlan 類別


steps=['寫報告', '審核報告', '發送電子郵件'] tools_needed=['Microsoft Word', 'Gmail'] estimated_time='2 小時'
<class '__main__.TaskPlan'>
